<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [5]:
## download data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-03-18 07:03:30--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  20.8MB/s    in 13s     

2020-03-18 07:03:44 (18.6 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
## unzip
!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
## imports
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.preprocessing.image import ImageDataGenerator

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [0]:
## init image generator parameters
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [0]:
## init data generator
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [8]:
## training data generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [9]:
## validation data generator
valid_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [10]:
## define keras model
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())
model.layers[0].trainable = False # make sure the VGG layer is not trained
print(model.summary()) # note the drop in trainable parameters from 14.7m to 1026





58892288/58889256 [==============================] - 6s 0us/step






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 1

In [11]:
## compile model with 'adam' optimizer and categorical crossentropy loss, according to instructions
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
## hyperparameters
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)
num_epochs = 2 # 

In [14]:
## train model
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
301/301 [==============================] - 370s 1s/step - loss: 0.0848 - acc: 0.9771 - val_loss: 0.0259 - val_acc: 0.9943
Epoch 2/2
301/301 [==============================] - 356s 1s/step - loss: 0.0204 - acc: 0.9956 - val_loss: 0.0154 - val_acc: 0.9963


In [0]:
## save model for later use
model.save('classifier_vgg16_model.h5')

In [16]:
## check if model was saved correctly
from keras.models import load_model
 
# load model
vgg16_model = load_model('classifier_vgg16_model.h5')
# summarize model.
vgg16_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


## 1.5 - Special section to recalculate ResNet from week 3

In [17]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
!unzip -q concrete_data_week3.zip

--2020-03-18 07:17:34--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip’

concrete_data_week3 100%[===================>] 249.37M  19.1MB/s    in 14s     

2020-03-18 07:17:49 (17.8 MB/s) - ‘concrete_data_week3.zip’ saved [261482368/261482368]



In [18]:
## imports
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.preprocessing.image import ImageDataGenerator

# load resNet specific packages
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

## init image generator parameters
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

# build data generators
data_generator_resNet = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator_resNet = data_generator_resNet.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

valid_generator_resNet = data_generator_resNet.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 10001 images belonging to 2 classes.


In [19]:
# build resNet model
resNet_model = Sequential()
resNet_model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
resNet_model.add(Dense(num_classes, activation='softmax'))
resNet_model.layers[0].trainable = False

resNet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# hyperparameters
steps_per_epoch_training = len(train_generator_resNet)
steps_per_epoch_validation = len(valid_generator_resNet)
num_epochs = 2 

# train model
fit_history = resNet_model.fit_generator(
    train_generator_resNet,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator_resNet,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)





94658560/94653016 [==============================] - 7s 0us/step
Epoch 1/2
301/301 [==============================] - 300s 996ms/step - loss: 0.0505 - acc: 0.9825 - val_loss: 0.2393 - val_acc: 0.8973
Epoch 2/2
301/301 [==============================] - 290s 964ms/step - loss: 0.0152 - acc: 0.9959 - val_loss: 0.3731 - val_acc: 0.8653


In [0]:
resNet_model.save('classifier_resnet_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [21]:
# load model
resNet_model = load_model('classifier_resnet_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [22]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

## validation data generator
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [27]:
print('Performance of resNet model on test data:')
resNet_eval = resNet_model.evaluate_generator(test_generator)
print('test_loss: '+str(resNet_eval[0])+' - test_acc: '+ str(resNet_eval[1]))

Performance of resNet model on test data:
test_loss: 0.35939953740336933 - test_acc: 0.8679999990463256


In [28]:
print('Performance of VGG16 model on test data:')
vgg16_eval = vgg16_model.evaluate_generator(test_generator)
print('test_loss: '+str(vgg16_eval[0])+' - test_acc: '+ str(vgg16_eval[1]))

Performance of VGG16 model on test data:
test_loss: 0.0127755619706586 - test_acc: 0.996


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [44]:
resNet_pred = resNet_model.predict_generator(test_generator)
resNet_labels = ['Positive' if(x[0]>0.5) else 'Negative' for x in resNet_pred]

print('ResNet predictions for images 1-5:')
print(resNet_labels[0:5])

ResNet predictions for images 1-5:
['Positive', 'Positive', 'Positive', 'Positive', 'Positive']


In [45]:
vgg16_pred = vgg16_model.predict_generator(test_generator)
vgg16_labels = ['Positive' if(x[0]>0.5) else 'Negative' for x in vgg16_pred]

print('VGG16 predictions for images 1-5:')
print(vgg16_labels[0:5])

VGG16 predictions for images 1-5:
['Positive', 'Positive', 'Positive', 'Positive', 'Positive']


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).